In [1]:
#imports
import pandas as pd
from sqlalchemy import create_engine, select, func, MetaData, Table, Column, Integer, String
from dotenv import load_dotenv
import os
from src.extract.nflreadpy_extract import *
from src.transform import cleaning, validation, fe_module
from src.load.load_module import DataLoader

#load_dotenv()
load = DataLoader()




['root', '127.0.0.1', 'DataPipelineMySQL8']


In [2]:
# EXTRACT DATA
teams = get_teams()
dataframes = {}
for year in range(1999, 2025):
    stats_df = get_team_stats(year)
    schedule_df = get_schedule(year)
    dataframes.setdefault(year, [stats_df, schedule_df])

In [3]:
# TRANSFORM DATA
team_table = fe_module.team_table(teams)
#print(team_table)
tables = {}
for year, items in dataframes.items():
    game_table = fe_module.game_table(items[1])
    season_table = fe_module.season_table(items[0])
    fact_table = fe_module.facts_table(stats_df = items[0], schedule_df = items[1])
    cleaned_fact = cleaning.cleaning.clean(fact_table)
    cleaned_season = cleaning.cleaning.clean(season_table)
    tables.setdefault(year, [cleaned_season, game_table, cleaned_fact])



   season_id  num_games
0       1999         16
     season_id  week team_id season_type opponent_team  completions  attempts  \
0         1999     1     ARI         REG           PHI           25        48   
1         1999     1     ATL         REG           MIN           18        31   
2         1999     1     BUF         REG           IND           22        42   
3         1999     1     CAR         REG            NO           16        32   
4         1999     1     CHI         REG            KC           31        47   
..         ...   ...     ...         ...           ...          ...       ...   
512       1999    20     STL        POST            LA           26        43   
513       1999    20      TB        POST            LA           13        29   
514       1999    20     TEN        POST           JAX           14        23   
515       1999    21     STL        POST            LA           24        45   
516       1999    21     TEN        POST            LA       

In [4]:
# LOAD DATA
load.create_(df = team_table, table_name = 'team', primary_key = 'team_id')
load.insert_(df = team_table, table_name= 'team', primary_key = 'team_id')

load.create_(df = tables[1999][0], table_name = 'season', primary_key = 'season_id')
load.create_(df = tables[1999][1], table_name = 'game', primary_key = 'game_id')
load.create_(df = tables[1999][2], table_name = 'nfl_facts', primary_key = 'id')

for items in tables.values():
    print(items[0])
    load.insert_(df = items[0], table_name= 'season', primary_key = 'season_id')
    load.insert_(df = items[1], table_name= 'game', primary_key = 'game_id')
    load.insert_(df = items[2], table_name= 'nfl_facts', primary_key = 'id')


2026-02-23 09:35:09,011 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2026-02-23 09:35:09,012 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-02-23 09:35:09,013 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2026-02-23 09:35:09,013 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-02-23 09:35:09,014 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2026-02-23 09:35:09,015 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-02-23 09:35:09,016 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-23 09:35:09,016 INFO sqlalchemy.engine.Engine DESCRIBE `datapipelinemysql8`.`team`
2026-02-23 09:35:09,016 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-02-23 09:35:09,018 INFO sqlalchemy.engine.Engine ROLLBACK
2026-02-23 09:35:09,018 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-23 09:35:09,019 INFO sqlalchemy.engine.Engine CREATE TABLE team (`team_id` VARCHAR(50) PRIMARY KEY, `team_name` VARCHAR(50), `team_conf` VARCHAR(50), `team_division` VARCHAR(50));
2026-02-23 09:35:0